# Problema de Merton: alocación óptima en un portafolio

#### Nicolás Romero Díaz
#### Maestría en Finanzas Cuantitativas

## Modelo de mercado

Nuestro mercado consiste en dos activos. Un activo riesgoso $S_t$ y un activo libre de riesgo $B_t$:

$$dS_t = S_t(\mu dt + \sigma dW_t)$$
$$dB_t = rB_tdt$$

Invertimos una proporción de capital $\pi$ en el activo $S_t$, de tal forma tenemos que el proceso de precios de nuestro portafolio $$X_t = S_t + B_t$$ será:
$$dX_t = X_t \left[ (r-\pi (\mu -r))dt + \pi\sigma dW_t \right]$$

Asumimos que el inversionista tiene una función de utilidad hiperbólica HARA de la forma:

$$U(x)=\frac{x^{\gamma}}{\gamma} \;\ \gamma\in(0, 1)$$

## Método Monte-Carlo

Asumiendo una estrategia $\pi$ constante, simulamos varios caminos para el proceso de riqueza $V(t, x)$. Discretizando la estrategia $\pi_i \in [0, 1]$ simulamos caminos para $X_t$. El objetivo del inversionsta es:

$$\max_{\pi_t}\left\{ \mathbb{E}\left[ U(X_T^{\pi}) | X_t = x \right] \right\}$$

Tras tener los caminos de $X_t$ para cada estrategia, aplicamos Monte-Carlo en la esperanza para maximizar sobre todos los valores de $U(X_T^{\pi})$

Se utilizan los siguientes parámetros para la simulación:

 * $r=0.06$
 * $\mu = 0.1$
 * $\sigma = 0.35$
 * $X_0 = 100$
 * $T=1$
 * $\gamma = 0.25$

## Aplicación

El valor del portafolio sigue la ecuación:

$$X_t = X_0 \exp \left\{ \left[  r-\pi(\mu-r) -\frac{1}{2}\pi^2\sigma^2\right]T + \pi\sigma\sqrt{T}\ Z\right\}$$

$$Z\sim \mathcal{N}(0, 1)$$

In [1]:
import numpy as np
import random as rd
import matplotlib.pyplot as plt

In [2]:
# Definimos los parametros

pmt = {'r': 0.06, 'mu': 0.1, 'sigma': 0.35, 'X0': 100., 'T': 1., 'gamma': 0.25}

In [3]:
# Definimos la funcion de drift

def drift(param, prop):
    return param['r'] + prop*(param['mu'] - param['r']) - 0.5*(param['sigma']*param['sigma']*prop*prop)

In [4]:
# Simulamos el precio del portafolio

def PortfolioPrice(param, N_steps, prop):
    p = [0.0]*(N_steps)
    p[0] = param['X0']
    
    for i in range(1, N_steps):
        p[i] = param['X0']*np.exp(drift(param, prop)+ prop*param['sigma']*np.sqrt(param['T'])*rd.normalvariate(0, 1))
        
    return p

In [5]:
# Definimos nuestra funcion de utilidad

def Utility(param, x):
    return np.power(x, param['gamma'])/param['gamma']

In [11]:
# Generamos N_path simulaciones del valor del portafolio

# Esta funcion retorna una lista que contiene la proporcion, la riqueza final media y la utilidad media
# dada una proporcion pi y simulando un numero N_path de caminos

def GeneratePaths(param, N_path, N_steps, prop):
    #path = []
    U = []
    XT = []
    
    for i in range(N_path):
        temp = PortfolioPrice(param, N_steps, prop)
        XT.append(temp[-1])
        #path.append(temp)
        U.append(Utility(param, XT[i]))
        
    return prop, np.mean(XT), np.mean(U) #path, np.mean(U)

In [12]:
# Definimos la funcion que nos calcula la utilidad esperada para cada pi

def MaximumProportion(param, N_path, N_steps):
    proportion = [i/100 for i in range(101)]
    result = []
    for i in range(len(proportion)):
        temp = GeneratePaths(param, N_path, N_steps, proportion[i])
        result.append(temp)
    return result

Ahora simulamos N_path$ = 500$ simulaciones para múltiples estrategias: $$\pi = [0.00, 0.01, 0.02, \dots, 0.99, 1.00]$$

In [17]:
N_paths = 10000
N_step  = 100

In [18]:
test = MaximumProportion(pmt, N_paths, N_step)

In [19]:
test

[(0.0, 106.18365465453597, 12.840277467117396),
 (0.01, 106.22487566753919, 12.84150863780804),
 (0.02, 106.25984949684077, 12.842520654118871),
 (0.03, 106.31379827005658, 12.84407630727334),
 (0.04, 106.33520096224963, 12.844622588543512),
 (0.05, 106.36788778878024, 12.845471292403136),
 (0.06, 106.46664997789166, 12.848286305665908),
 (0.07, 106.5354426547202, 12.850186461209161),
 (0.08, 106.53362168937164, 12.849893288662656),
 (0.09, 106.56842129966272, 12.85070532376627),
 (0.1, 106.64272816570886, 12.852676949519127),
 (0.11, 106.69795398324617, 12.85396992987204),
 (0.12, 106.69605217401502, 12.853653294099168),
 (0.13, 106.67846726339012, 12.852656057232981),
 (0.14, 106.7085528645234, 12.853232602093875),
 (0.15, 106.79998890239946, 12.855478466745758),
 (0.16, 106.90543023635485, 12.85828038710107),
 (0.17, 106.89629320594659, 12.857520809496235),
 (0.18, 107.10911125086402, 12.863388359995469),
 (0.19, 106.97143748888909, 12.858689360443126),
 (0.2, 106.97313030219827, 12